In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error



In [2]:
data = pd.read_pickle("data/ready_dataset.pickle")
print(type(data))
test_data = data.iloc[:40]
train_data = data.iloc[40:]

<class 'pandas.core.frame.DataFrame'>


In [3]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        sample["grace_value"] = [[sample["grace_value"]], [0]]
        row = np.concatenate((sample["evap_value"], sample["lvl1_value"], sample["lvl2_value"], sample["lvl3_value"], sample["lvl4_value"], sample["precip_value"], sample["grace_value"]), axis=1)
        row_concat = np.concatenate((row[0], row[1])) 
        x = torch.tensor(row_concat, dtype=torch.float32)       
        y = torch.tensor(sample['target'], dtype=torch.float32)
        return x, y


In [4]:
my_dataset = MyDataset(train_data)

In [5]:
#my_dataset.__getitem__(0)

In [99]:

class MyLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyLSTM, self).__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size)
        self.fc = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x.unsqueeze(1))
        x = self.fc(x[:, -1, :])
        return x.squeeze()


# konwersja danych na tensor i załadowanie do DataLoader


def collate_fn(batch):
    x = [item[0] for item in batch]
    y = [item[1] for item in batch]
    x = torch.nn.utils.rnn.pad_sequence(x, batch_first=True)
    y = torch.stack(y)
    return x, y

def collate_fn_lstm(batch):
    """
    Funkcja collate dla sekwencji wejściowych dla sieci LSTM.
    """
    # Sortujemy batch względem długości sekwencji wejściowych
    batch = sorted(batch, key=lambda x: x[0].shape[0], reverse=True)
    
    # Tworzymy mini-batche z wyrównaniem długości sekwencji wejściowych
    inputs = [torch.FloatTensor(item[0]) for item in batch]
    inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=False)
    
    # Tworzymy mini-batche z etykietami
    targets = [torch.LongTensor(item[1]) for item in batch]
    targets = torch.nn.utils.rnn.pad_sequence(targets, batch_first=False)
    
    return inputs, targets


batch_size = 4
my_dataloader = DataLoader(my_dataset, batch_size=batch_size, shuffle=True) ## collate_fn=collate_fn)

# inicjalizacja modelu i uruchomienie treningu
model = MyLSTM(input_size=390, hidden_size=50, output_size=1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):
    for batch_x, batch_y in my_dataloader:
        optimizer.zero_grad()
        y_pred = model(batch_x)
        loss = criterion(y_pred, batch_y)
        loss.backward()
        optimizer.step()

/tmp/ipykernel_184/1931264250.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["grace_value"] = [[sample["grace_value"]], [0]]
/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [100]:
test_dataset = MyDataset(test_data)

In [101]:
test_dataloader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=False)

y_true = []
y_pred = []
model.eval()
with torch.no_grad():
    for batch_x, batch_y in test_dataloader:
        y_true += batch_y.tolist()
        y_pred += model(batch_x).tolist()

mse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)

print(f"MSE: {mse:.4f}")
print(f"R^2: {r2:.4f}")
print(f"MAE: {mae:.4f}")

MSE: 0.0368
R^2: -1.2025
MAE: 0.1276


/tmp/ipykernel_184/1931264250.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["grace_value"] = [[sample["grace_value"]], [0]]
